# Libraries

In [ ]:
!pip install dalle-pytorch --upgrade

Defaulting to user installation because normal site-packages is not writeable
  Using cached dalle_pytorch-1.0.6-py3-none-any.whl (1.4 MB)
  Using cached g_mlp_pytorch-0.1.5-py3-none-any.whl (5.9 kB)
  Attempting uninstall: dalle-pytorch
    Found existing installation: dalle-pytorch 0.3.5
    Uninstalling dalle-pytorch-0.3.5:
      Successfully uninstalled dalle-pytorch-0.3.5
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [ ]:
!pip install wandb
import wandb
!wandb login

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 97 kB 6.4 MB/s 
     |████████████████████████████████| 170 kB 44.3 MB/s 
     |████████████████████████████████| 133 kB 48.3 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=3680d8a57d3f6a407e09ab308effb5005fc0bbc0e65420af05d726e496828796
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=00943ea9680e379bdfbd0c64227fa3141fde82a43d87ba053ebc166048deb1aa
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
Abo

In [ ]:
!pip install gdown

# Dataset

In [ ]:
!gdown https://drive.google.com/uc?id=1P189d4J3SBaBzevBToNegnVNaKeIIeej

In [ ]:
!tar xvf Danbooru2017_CUB.tar.gz
!mv Danbooru2017_CUB CUB_200_2011

In [ ]:
os.listdir("./CUB_200_2011/")

['classes.txt',
 'images',
 'image_class_labels.txt',
 'parts',
 'train_test_split.txt',
 'bounding_boxes.txt',
 'README',
 'attributes',
 'images.txt']

# Tokenizer

In [ ]:
from random import choice
from pathlib import Path
import os
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer  

IMAGE_TEXT_FOLDER = os.path.realpath('../AttnGAN/data/anime_wb_full/text/')

path = Path(IMAGE_TEXT_FOLDER)
text_files = [*path.glob('**/*.txt')]
text_files = {t.stem: t for t in text_files}

text_desc = []

for text_file, text_file_posix in text_files.items():

    descriptions = open(text_file_posix, "r").readlines()[0].strip()
    descriptions = descriptions.split(" ")
    text_desc.append(descriptions)

tokenizer = Tokenizer(oov_token='OOV')
tokenizer.fit_on_texts(text_desc)

In [ ]:
print(tokenizer.word_index)
len(tokenizer.word_index)

{'OOV': 1, '1girl': 2, 'white_background': 3, 'solo': 4, 'simple_background': 5, 'long_hair': 6, 'looking_at_viewer': 7, 'smile': 8, 'blush': 9, 'breasts': 10, 'short_hair': 11, 'skirt': 12, 'open_mouth': 13, 'full_body': 14, 'blue_eyes': 15, 'thighhighs': 16, 'blonde_hair': 17, 'bangs': 18, 'brown_hair': 19, 'ribbon': 20, 'red_eyes': 21, 'dress': 22, 'gloves': 23, 'long_sleeves': 24, 'large_breasts': 25, 'hat': 26, 'hair_ornament': 27, 'bow': 28, 'original': 29, 'cleavage': 30, 'black_hair': 31, 'shirt': 32, 'bare_shoulders': 33, 'very_long_hair': 34, 'brown_eyes': 35, 'school_uniform': 36, 'medium_breasts': 37, 'sitting': 38, 'twintails': 39, 'eyebrows_visible_through_hair': 40, 'black_legwear': 41, 'navel': 42, 'standing': 43, 'upper_body': 44, 'animal_ears': 45, 'holding': 46, 'green_eyes': 47, 'jewelry': 48, 'boots': 49, 'hair_ribbon': 50, 'weapon': 51, 'collarbone': 52, 'underwear': 53, 'purple_eyes': 54, 'shoes': 55, 'pleated_skirt': 56, 'd': 57, 'short_sleeves': 58, 'hair_betwe

7480

In [ ]:
import random

def my_tokenize(string, pad=50, randomize=True):

    seq = []

    for word in string.strip().split(" "):

        if word in tokenizer.word_index:
            seq.append(tokenizer.word_index[word])

    if randomize:
        random.shuffle(seq)

    while len(seq) < pad:
        seq.append(0)

    return np.array(seq[0:pad])

def decode_text(padded):

    words = []

    for index in padded:

        if index == 0:
            continue

        for key, value in tokenizer.word_index.items():

            if value == index:
                words.append(key)
                break

    return " ".join(words)
    

padded_train = my_tokenize("breasts long_pointy_ears twintails X", pad=10)
print(padded_train)
print(padded_train != 0)
print(decode_text(padded_train))


[  10   39 2897    0    0    0    0    0    0    0]
[ True  True  True False False False False False False False]
breasts twintails long_pointy_ears


# Training

In [ ]:
from random import choice
from pathlib import Path
import os

# torch

import torch
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_

# vision imports

from PIL import Image
from torchvision import transforms as T
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid, save_image

# dalle related classes and utils

from dalle_pytorch import OpenAIDiscreteVAE, DiscreteVAE, DALLE

# helpers

def exists(val):
    return val is not None

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# argument parsing

VAE_PATH = './anime_wb_100_ep_220/vae.pt' # - will use OpenAIs pretrained VAE if not set
DALLE_PATH = None # './anime_wb_100_ep_220/dalle.pt' # None # './dalle.pt'
TAMING = False    # use VAE from taming transformers paper
IMAGE_TEXT_FOLDER = os.path.realpath('../AttnGAN/data/anime_wb_full/')
BPE_PATH = None
RESUME = exists(DALLE_PATH)

EPOCHS = 300
BATCH_SIZE = 4
LEARNING_RATE = 3e-4
GRAD_CLIP_NORM = 0.5

MODEL_DIM = 512
TEXT_SEQ_LEN = 100
DEPTH = 2
HEADS = 4
DIM_HEAD = 64
REVERSIBLE = True

# tokenizer
VOCAL_SIZE = len(tokenizer.word_counts) + TEXT_SEQ_LEN

# reconstitute vae

if RESUME:
    dalle_path = Path(DALLE_PATH)
    assert dalle_path.exists(), 'DALL-E model file does not exist'

    loaded_obj = torch.load(str(dalle_path))

    dalle_params, vae_params, weights = loaded_obj['hparams'], loaded_obj['vae_params'], loaded_obj['weights']

    if vae_params is not None:
        vae = DiscreteVAE(**vae_params)
    else:
        vae = OpenAIDiscreteVAE()

    dalle_params = dict(        
        **dalle_params
    )

    IMAGE_SIZE = vae.image_size

else:
    if exists(VAE_PATH):

        vae_path = Path(VAE_PATH)
        assert vae_path.exists(), 'VAE model file does not exist'

        loaded_obj = torch.load(str(vae_path))

        vae_params, weights = loaded_obj['hparams'], loaded_obj['weights']

        vae = DiscreteVAE(**vae_params)
        vae.load_state_dict(weights)
    else:
        print('using pretrained VAE for encoding images to tokens')
        vae_params = None

        vae_klass = OpenAIDiscreteVAE if not TAMING else VQGanVAE1024
        vae = vae_klass()

    IMAGE_SIZE = vae.image_size

    dalle_params = dict(
        num_text_tokens = VOCAL_SIZE,
        text_seq_len = TEXT_SEQ_LEN,
        dim = MODEL_DIM,
        depth = DEPTH,
        heads = HEADS,
        dim_head = DIM_HEAD,
        reversible = REVERSIBLE,
        attn_types = ('full', 'axial_row', 'axial_col', 'conv_like')
    )

# helpers

def save_model(path):
    save_obj = {
        'hparams': dalle_params,
        'vae_params': vae_params,
        'weights': dalle.state_dict()
    }

    torch.save(save_obj, path)

# dataset loading

class TextImageDataset(Dataset):
    def __init__(self, folder, text_len = 256, image_size = 128):
        super().__init__()
        path = Path(folder)

        text_files = [*path.glob('**/*.txt')]

        image_files = [
            *path.glob('**/*.png'),
            *path.glob('**/*.jpg'),
            *path.glob('**/*.jpeg')
        ]

        text_files = {t.stem: t for t in text_files}
        image_files = {i.stem: i for i in image_files}

        keys = (image_files.keys() & text_files.keys())

        self.keys = list(keys)
        self.text_files = {k: v for k, v in text_files.items() if k in keys}
        self.image_files = {k: v for k, v in image_files.items() if k in keys}

        self.image_tranform = T.Compose([
            T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
            T.RandomResizedCrop(image_size, scale = (0.75, 1.), ratio = (1., 1.)),
            T.ToTensor()
        ])

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, ind):
        key = self.keys[ind]
        text_file = self.text_files[key]
        image_file = self.image_files[key]

        image = Image.open(image_file)
        #descriptions = text_file.read_text().split('\n')
        #descriptions = list(filter(lambda t: len(t) > 0, descriptions))
        #description = choice(descriptions)

        with open(text_file, "r") as f:
            seq = f.readlines()[0].strip()
            tokenized_text = my_tokenize(seq, pad=TEXT_SEQ_LEN)
            mask = tokenized_text != 0

        #seq = tokenizer.texts_to_sequences([description])
        #tokenized_text = pad_sequences(seq, maxlen = TEXT_SEQ_LEN, padding = 'post')[0]
        #tokenized_text = my_tokenize()
        #mask = tokenized_text != 0

        image_tensor = self.image_tranform(image)
        return tokenized_text, image_tensor, mask

# create dataset and dataloader

ds = TextImageDataset(
    IMAGE_TEXT_FOLDER,
    text_len = TEXT_SEQ_LEN,
    image_size = IMAGE_SIZE
)

assert len(ds) > 0, 'dataset is empty'
print(f'{len(ds)} image-text pairs found for training')

dl = DataLoader(ds, batch_size = BATCH_SIZE, shuffle = True, drop_last = True)

# initialize DALL-E

dalle = DALLE(vae = vae, **dalle_params).cuda()

if RESUME:
    dalle.load_state_dict(weights)

# optimizer

opt = Adam(dalle.parameters(), lr = LEARNING_RATE)

# experiment tracker

import wandb

model_config = dict(
    depth = DEPTH,
    heads = HEADS,
    dim_head = DIM_HEAD
)

run = wandb.init(project = 'dalle_train_transformer', resume = RESUME, config = model_config)

# training

for epoch in range(EPOCHS):
    for i, (text, images, mask) in enumerate(dl):
        text, images, mask = map(lambda t: t.cuda(), (text, images, mask))

        loss = dalle(text, images, mask = mask, return_loss = True)

        loss.backward()
        clip_grad_norm_(dalle.parameters(), GRAD_CLIP_NORM)

        opt.step()
        opt.zero_grad()

        log = {}

        if i % 10 == 0:
            print(epoch, i, f'loss - {loss.item()}')

            log = {
                **log,
                'epoch': epoch,
                'iter': i,
                'loss': loss.item()
            }

        if i % 1000 == 0:
            sample_text = text[:1]
            token_list = sample_text.masked_select(sample_text != 0).tolist()
            decoded_text = decode_text(token_list)

            image = dalle.generate_images(
                text[:1],
                mask = mask[:1],
                filter_thres = 0.9    # topk sampling at 0.9
            )

            save_model(f'./dalle.pt')
            wandb.save(f'./dalle.pt')

            log = {
                **log,
                'image': wandb.Image(image, caption = decoded_text)
            }

        wandb.log(log)

    # save trained model to wandb as an artifact every epoch's end

    model_artifact = wandb.Artifact('trained-dalle', type = 'model', metadata = dict(model_config))
    model_artifact.add_file('dalle.pt')
    run.log_artifact(model_artifact)

save_model(f'./dalle-final.pt')
wandb.save('./dalle-final.pt')
model_artifact = wandb.Artifact('trained-dalle', type = 'model', metadata = dict(model_config))
model_artifact.add_file('dalle-final.pt')
run.log_artifact(model_artifact)

wandb.finish()


14453 image-text pairs found for training


wandb: W&B syncing is set to `offline` in this directory.  Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.


0 0 loss - 9.277132987976074
0 10 loss - 5.959808349609375
0 20 loss - 3.506715774536133
0 30 loss - 2.454176187515259
0 40 loss - 2.893954277038574
0 50 loss - 2.191697835922241
0 60 loss - 2.136192798614502
0 70 loss - 2.3646109104156494
0 80 loss - 1.8515923023223877
0 90 loss - 2.1615488529205322
0 100 loss - 2.116283655166626
0 110 loss - 2.272986888885498
0 120 loss - 2.7789223194122314
0 130 loss - 2.259046792984009
0 140 loss - 2.1573538780212402
0 150 loss - 2.6952545642852783
0 160 loss - 2.4304513931274414
0 170 loss - 2.0836734771728516
0 180 loss - 1.944146752357483
0 190 loss - 2.339047908782959
0 200 loss - 1.935229778289795
0 210 loss - 2.004563093185425
0 220 loss - 1.9256055355072021
0 230 loss - 2.4952147006988525
0 240 loss - 2.181318759918213
0 250 loss - 2.1378414630889893
0 260 loss - 1.9706882238388062
0 270 loss - 2.2072901725769043
0 280 loss - 2.569707155227661
0 290 loss - 2.1079177856445312
0 300 loss - 1.843839168548584
0 310 loss - 2.185731887817383
0 320

/workspace/home/jesandov/.local/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:793: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


Streaming output truncated to the last 5000 lines.
286 680 loss - 0.8973888158798218
286 690 loss - 1.0955572128295898
286 700 loss - 1.5374457836151123
286 710 loss - 1.2389516830444336
286 720 loss - 1.2988641262054443
286 730 loss - 0.8435247540473938
286 740 loss - 1.405513048171997
286 750 loss - 1.363210678100586
286 760 loss - 1.0794316530227661
286 770 loss - 1.2574976682662964
286 780 loss - 1.259017825126648
286 790 loss - 1.0894546508789062
286 800 loss - 1.083931803703308
286 810 loss - 1.2283803224563599
286 820 loss - 1.2039791345596313
286 830 loss - 1.3769994974136353
286 840 loss - 1.1823610067367554
286 850 loss - 1.4174911975860596
286 860 loss - 1.2649548053741455
286 870 loss - 0.9503085017204285
286 880 loss - 1.5053764581680298
286 890 loss - 1.5800880193710327
286 900 loss - 1.1765029430389404
286 910 loss - 1.2774962186813354
286 920 loss - 1.3253204822540283
286 930 loss - 1.1943488121032715
286 940 loss - 1.3299055099487305
286 950 loss - 1.0091389417648315
2

epoch,299
iter,3610
loss,1.60107
_runtime,712145
_timestamp,1631008417
_step,1083897


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
iter,▇▄█▄▁▅▁▆▂▆▃▇▃▇▄█▄▁▅▁▆▂▆▃▇▃▇▄█▄▁▅▁▆▂▆▃▇▃▇
loss,▄▅▆▄▃▄▂▃▄▃▅▄▃▅▃▂▂▅▃▂▅▃▅▅▅▆▃▃█▃▂▅▄▂▄▁▃▂▂▄
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


wandb: You can sync this run to the cloud by running:
wandb: wandb sync /workspace/home/jesandov/Code/Tesis/dalle-pytorch/wandb/offline-run-20210829_230431-3lb8ck9t


# Sample

In [ ]:
import time
import wandb
import torch
import matplotlib.pyplot as plt

from random import choice
from pathlib import Path
import os

import torch
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_

from PIL import Image
from torchvision import transforms as T
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid, save_image

from dalle_pytorch import OpenAIDiscreteVAE, DiscreteVAE, DALLE

def load_dalle(prefix="."):

    VAE_PATH = os.path.realpath(f'{prefix}/vae.pt')
    DALLE_PATH = os.path.realpath(f'{prefix}/dalle.pt')

    # reconstitute vae
    dalle_path = Path(DALLE_PATH)
    assert dalle_path.exists(), f'DALL-E model file "{dalle_path}" does not exist'

    loaded_obj = torch.load(str(dalle_path))

    dalle_params, vae_params, weights = loaded_obj['hparams'], loaded_obj['vae_params'], loaded_obj['weights']

    if vae_params is not None:
        vae = DiscreteVAE(**vae_params)
    else:
        vae = OpenAIDiscreteVAE()

    dalle_params = dict(        
        **dalle_params
    )

    dalle = DALLE(vae = vae, **dalle_params).cuda()
    dalle.load_state_dict(weights)

    return dalle


def pretty_time(sec):

    if sec < 60:
        return "{0} sec".format(sec)

    elif sec < (60 * 60):

        module = sec % (60)
        divide = int(sec / (60))
        return "{0} min {1}".format(divide, pretty_time(module))

    else:

        module = sec % (60 * 60)
        divide = int(sec / (60 * 60))
        return "{0} hrs {1}".format(divide, pretty_time(module))

def time_decorator(func):
    def wrapper(*args, **kwargs):

        print("*"*50)

        start_time = time.time()

        res = func(*args, **kwargs)

        end_time = time.time()
        total_time = end_time - start_time

        print("{0} executed in {1}".format(func, pretty_time(total_time)))
        print("*"*50)
        return res
    return wrapper


@time_decorator
def sample_image(text, prefix, text_size):

    run = wandb.init(project = 'dalle_train_transformer')

    try:
        text_example = my_tokenize(text, pad=text_size)
        mask_example = text_example != 0

        text_example = torch.from_numpy(np.array([text_example])).cuda()
        mask_example = torch.from_numpy(np.array([mask_example])).cuda()

        dalle = load_dalle(prefix)

        image = dalle.generate_images(
            text_example,
            mask = mask_example,
            filter_thres = 0.9    # topk sampling at 0.9
        )

        wand_image = wandb.Image(image)
        return wandb.Image(image)._image
    
    finally:
        wandb.finish()
    

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import keras.backend as K

#import tensorflow.compat.v1.keras.backend as K
#import tensorflow as tf
#tf.compat.v1.disable_eager_execution()

@time_decorator
def generate_images_samples():

    opt = {
        #"anime_wb_40_ep_50": 40,
        #"anime_wb_40_ep_100": 40,
        #"anime_wb_50": 50,
        #"anime_wb_100_ep_200": 100,
        #"anime_wb_100_ep_220": 100,
        #"anime_wb_100_ep_300": 100,
        #"anime_wb_sep_200": 256
        "anime_wb_100_ep_300_v2": 100
    }

    # Generate graph
    font_size = 15
    plt.rcParams['axes.facecolor']='gray'
    plt.rcParams['savefig.facecolor']='gray'
    plt.rcParams.update({'font.size': font_size})

    originals = os.listdir("original_wb")

    for i in range(0, 10):

        os.system(f"mkdir -p sample{i}")

        for original in originals:

            if original.endswith(".txt"):

                orig = original.replace(".txt", "")

                text = ""
                with open(f"original_wb/{orig}.txt", "r") as _f:
                    text = _f.readlines()[0].strip()
                    
                orig_img = load_img(f"original_wb/{orig}.jpg")
                images_array = [{"prefix": "original", "image": orig_img}]

                for prefix, text_size in opt.items():
                    image = sample_image(text, prefix, text_size)
                    images_array.append({"prefix": prefix, "image": image})
                    image.save(f"sample{i}/{orig}.png")

                continue
                r, c = 2, int(len(images_array)/2)

                plt.rcParams["figure.figsize"] = (c*4,r*4)
                fig, axs = plt.subplots(r, c)
                fig.suptitle(original, fontsize=font_size*1.5)

                cnt = 0
                for i in range(r):
                    for j in range(c):

                        img = images_array[cnt]["image"]
                        axs[i,j].imshow(img)
                        axs[i,j].set_title(images_array[cnt]["prefix"])

                        axs[i,j].axis('off')
                        cnt += 1

                os.system("mkdir -p result")
                fig.savefig("{0}.png".format(os.path.join("result", orig)))
                plt.show()


generate_images_samples()

**************************************************
**************************************************


wandb: W&B syncing is set to `offline` in this directory.  Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.


wandb: You can sync this run to the cloud by running:
wandb: wandb sync /workspace/home/jesandov/Code/Tesis/dalle-pytorch/wandb/offline-run-20210907_184903-7qb0epcg


<function sample_image at 0x7f5a16f95d08> executed in 1 min 13.994925498962402 sec
**************************************************
**************************************************


wandb: W&B syncing is set to `offline` in this directory.  Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.


...

wandb: You can sync this run to the cloud by running:
wandb: wandb sync /workspace/home/jesandov/Code/Tesis/dalle-pytorch/wandb/offline-run-20210907_204937-3s8ib5r3


<function sample_image at 0x7f5a16f95d08> executed in 1 min 13.476890802383423 sec
**************************************************
**************************************************


wandb: W&B syncing is set to `offline` in this directory.  Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.


wandb: You can sync this run to the cloud by running:
wandb: wandb sync /workspace/home/jesandov/Code/Tesis/dalle-pytorch/wandb/offline-run-20210907_205050-3uicq5ue


<function sample_image at 0x7f5a16f95d08> executed in 1 min 13.428171873092651 sec
**************************************************
<function generate_images_samples at 0x7f5a18433e18> executed in 2 hrs 3 min 0.8222846984863281 sec
**************************************************
